In [ ]:
# Libraries

import pandas as pd
import numpy as np


from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet

In [ ]:
train = pd.read_excel("data_train.xlsx")
test = pd.read_excel("data_test.xlsx")
sample = pd.read_excel("Sample_Submission.xlsx")

In [ ]:
train.head(50)


In [ ]:
print(train.info())


In [ ]:
train.describe(include = 'all').head(2)


In [6]:
train = train[['Title', 'Author', 'Edition', 'Reviews', 'Ratings','Genre',
               'BookCategory', 'Price']]
test = test[['Title', 'Author', 'Edition', 'Reviews', 'Ratings','Genre',
               'BookCategory']]


In [7]:
train.head()


,Title,Author,Edition,Reviews,Ratings,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,Photography Textbooks,"Arts, Film & Photography",965.62


In [8]:
train.isnull().sum()


Title           0
Author          0
Edition         0
Reviews         0
Ratings         0
Genre           0
BookCategory    0
Price           0
dtype: int64

In [9]:
#Identifying the maximum number of authors for a single book from the given datasets
authors_1 = list(train['Author'])
authors_2 = list(test['Author'])

authors_1.extend(authors_2)

authorslis = [i.split(",") for i in authors_1]

max = 1
for i in authorslis:
  if len(i) >= max:
    max = len(i)
print("Max. number of authors for a single boook = ",max)

for i in range(len(authorslis)):
  if len(authorslis[i]) == max:
    print(i)    
    
all_authors = [author.strip().upper() for listin in authorslis for author in listin]


Max. number of authors for a single boook =  7
7008


In [10]:
def split_authors(data):
  
  authors = list(data)
  
  A1 = []
  A2 = []
  A3 = []
  A4 = []
  A5 = []
  A6 = []
  A7 = []
  for i in authors:
    
    try :
      A1.append(i.split(',')[0].strip().upper())
    except :
      A1.append('NONE')
      
    try :
      A2.append(i.split(',')[1].strip().upper())
    except :
      A2.append('NONE')
        
    try :
      A3.append(i.split(',')[2].strip().upper())
    except :
      A3.append('NONE')
        
    try :
      A4.append(i.split(',')[3].strip().upper())
    except :
      A4.append('NONE')
        
    try :
      A5.append(i.split(',')[4].strip().upper())
    except :
      A5.append('NONE')
      
    try :
      A6.append(i.split(',')[5].strip().upper())
    except :
      A6.append('NONE')
     
    try :
      A7.append(i.split(',')[6].strip().upper())
    except :
      A7.append('NONE')

      
  return A1,A2,A3,A4,A5,A6,A7
  
all_authors.append('NONE')


In [11]:
genre_1 = list(train['Genre'])
genre_2 = list(test['Genre'])

genre_1.extend(genre_2)

genre_lis = [i.split(",") for i in genre_1]


max = 1
for i in genre_lis:
  if len(i) >= max:
    max = len(i)
print("Max. number of genres for a single boook = ",max)
      
all_genres = [genre.strip().upper() for listin in genre_lis for genre in listin]


Max. number of genres for a single boook =  2


In [12]:
def split_genres(data):
  
  genres = list(data)
  
  G1 = []
  G2 = []
  
  for i in genres:
    
    try :
      G1.append(i.split(',')[0].strip().upper())
      
    except :
      G1.append('NONE')
      
    try :
      G2.append(i.split(',')[1].strip().upper())
    except :
      G2.append('NONE')


      
  return G1,G2
  
all_genres.append('NONE')

In [13]:
cat_1 = list(train['BookCategory'])
cat_2 = list(test['BookCategory'])

cat_1.extend(cat_2)

cat_lis = [i.split(",") for i in cat_1]


max = 1
for i in cat_lis:
  if len(i) >= max:
    max = len(i)
print("Max. number of Categories for a single boook = ",max)

all_categories = [cat.strip().upper() for listin in cat_lis for cat in listin]


Max. number of Categories for a single boook =  2


In [14]:
def split_categories(data):
  
  cat = list(data)
  
  C1 = []
  C2 = []

  for i in cat:
    
    try :
      C1.append(i.split(',')[0].strip().upper())
    except :
      C1.append('NONE')
      
    try :
      C2.append(i.split(',')[1].strip().upper())
    except :
      C2.append('NONE')


      
  return C1,C2
  
all_categories.append('NONE')


In [15]:
def split_edition(data):  
  
  edition  = list(data)
  
  ed_type = [i.split(",– ")[0].strip().upper() for i in edition]
  
  edit_date = [i.split(",– ")[1].strip() for i in edition]
  
  m_y = [i.split()[-2:] for i in edit_date]
  
  for i in range(len(m_y)):
    if len(m_y[i]) == 1:
      m_y[i].insert(0,'NA')
      
  # Based on the given dataset below is the list of possible values for Months
  
  months =  ['Apr','Aug','Dec','Feb', 'Jan', 'Jul','Jun','Mar','May','NA','Nov','Oct','Sep']
  
  ed_month = [m_y[i][0].upper() if m_y[i][0] in months else 'NA' for i in range(len(m_y))]
  ed_year = [int(m_y[i][1].strip()) if m_y[i][1].isdigit() else 0 for i in range(len(m_y))]
  
  return ed_type, ed_month, ed_year


In [16]:
import re


def restructure(data):
  
  
  titles = list(data['Title'])
  
  titles = [title.strip().upper() for title in titles]
  
  
  a1,a2,a3,a4,a5,a6,a7 = split_authors(data['Author']) 
  
  ed_type, ed_month, ed_year = split_edition(data['Edition'])
  

  ratings = list(data['Reviews'])
  ratings = [float(re.sub(" out of 5 stars", "", i).strip()) for i in ratings]
  
  
  reviews = list(data['Ratings'])
  
  plu = ' customer reviews'
  
  reviews = [re.sub(" customer reviews", "", i) if plu in i else re.sub(" customer review", "", i) for i in reviews  ]
  reviews = [int(re.sub(",", "", i).strip()) for i in reviews ]
  

  
  g1, g2 = split_genres(data['Genre'])
  
  c1,c2 = split_categories(data['BookCategory'])

  structured_data = pd.DataFrame({'Title': titles,
                                  'Author1': a1,
                                  'Author2': a2,
                                  'Author3': a3,
                                  'Author4': a4,
                                  'Author5': a5,
                                  'Author6': a6,
                                  'Author7': a7,
                                  'Edition_Type': ed_type,
                                  'Edition_Month': ed_month,
                                  'Edition_Year': ed_year,
                                  'Ratings': ratings,
                                  'Reviews': reviews,
                                  'Genre1': g1,
                                  'Genre2': g2,
                                  'Category1': c1,
                                  'Category2': c2
                                  
                               })
  
  return structured_data

In [17]:
restructure(train).head(3)


,Title,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Edition_Type,Edition_Month,Edition_Year,Ratings,Reviews,Genre1,Genre2,Category1,Category2
0,THE PRISONER'S GOLD (THE HUNTERS 3),CHRIS KUZNESKI,NONE,NONE,NONE,NONE,NONE,NONE,PAPERBACK,MAR,2016,4.0,8,ACTION & ADVENTURE (BOOKS),NONE,ACTION & ADVENTURE,NONE
1,GURU DUTT: A TRAGEDY IN THREE ACTS,ARUN KHOPKAR,NONE,NONE,NONE,NONE,NONE,NONE,PAPERBACK,NOV,2012,3.9,14,CINEMA & BROADCAST (BOOKS),NONE,BIOGRAPHIES,DIARIES & TRUE ACCOUNTS
2,LEVIATHAN (PENGUIN CLASSICS),THOMAS HOBBES,NONE,NONE,NONE,NONE,NONE,NONE,PAPERBACK,FEB,1982,4.8,6,INTERNATIONAL RELATIONS,NONE,HUMOUR,NONE


In [18]:
X_train = restructure(train)

Y_train = train.iloc[:, -1].values

X_test = restructure(test)

In [19]:
X_train.describe(include = 'all')


,Title,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Edition_Type,Edition_Month,Edition_Year,Ratings,Reviews,Genre1,Genre2,Category1,Category2
count,6237,6237,6237,6237,6237,6237,6237,6237,6237,6237,6237.000000,6237.000000,6237.000000,6237,6237,6237,6237
unique,5564,3633,264,73,21,5,1,1,19,13,NaN,NaN,NaN,345,27,11,6
top,A GAME OF THRONES (A SONG OF ICE AND FIRE),AGATHA CHRISTIE,NONE,NONE,NONE,NONE,NONE,NONE,PAPERBACK,OCT,NaN,NaN,NaN,ACTION & ADVENTURE (BOOKS),NONE,ACTION & ADVENTURE,NONE
freq,4,69,5929,6159,6214,6233,6237,6237,5193,639,NaN,NaN,NaN,947,5594,818,3297
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.101972,4.293202,35.984287,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116.821510,0.662501,149.995031,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.000000,4.000000,2.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.000000,4.400000,7.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.000000,4.800000,22.000000,NaN,NaN,NaN,NaN


In [20]:
X_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6237 entries, 0 to 6236
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Title          6237 non-null   object 
 1   Author1        6237 non-null   object 
 2   Author2        6237 non-null   object 
 3   Author3        6237 non-null   object 
 4   Author4        6237 non-null   object 
 5   Author5        6237 non-null   object 
 6   Author6        6237 non-null   object 
 7   Author7        6237 non-null   object 
 8   Edition_Type   6237 non-null   object 
 9   Edition_Month  6237 non-null   object 
 10  Edition_Year   6237 non-null   int64  
 11  Ratings        6237 non-null   float64
 12  Reviews        6237 non-null   int64  
 13  Genre1         6237 non-null   object 
 14  Genre2         6237 non-null   object 
 15  Category1      6237 non-null   object 
 16  Category2      6237 non-null   object 
dtypes: float64(1), int64(2), object(14)
memory usage: 82

In [21]:
X_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Title          1560 non-null   object 
 1   Author1        1560 non-null   object 
 2   Author2        1560 non-null   object 
 3   Author3        1560 non-null   object 
 4   Author4        1560 non-null   object 
 5   Author5        1560 non-null   object 
 6   Author6        1560 non-null   object 
 7   Author7        1560 non-null   object 
 8   Edition_Type   1560 non-null   object 
 9   Edition_Month  1560 non-null   object 
 10  Edition_Year   1560 non-null   int64  
 11  Ratings        1560 non-null   float64
 12  Reviews        1560 non-null   int64  
 13  Genre1         1560 non-null   object 
 14  Genre2         1560 non-null   object 
 15  Category1      1560 non-null   object 
 16  Category2      1560 non-null   object 
dtypes: float64(1), int64(2), object(14)
memory usage: 20

In [22]:
# Finding Unique items for all columns
def unique_items(list1, list2):
  a = list1
  b = list2
  a.extend(b)
  return list(set(a))


In [23]:
from sklearn.preprocessing import LabelEncoder

le_Title = LabelEncoder()
all_titles = unique_items(list(X_train.Title),list(X_test.Title))
le_Title.fit(all_titles)

le_Edition_Type = LabelEncoder()
all_etypes = unique_items(list(X_train.Edition_Type),list(X_test.Edition_Type))
le_Edition_Type.fit(all_etypes)


le_Edition_Month = LabelEncoder()
all_em = unique_items(list(X_train.Edition_Month),list(X_test.Edition_Month))
le_Edition_Month.fit(all_em)

le_Author = LabelEncoder()
all_Authors = list(set(all_authors))
le_Author.fit(all_Authors)

le_Genre = LabelEncoder()
all_Genres = list(set(all_genres))
le_Genre.fit(all_Genres)

le_Category = LabelEncoder()
all_Categories = list(set(all_categories))
le_Category.fit(all_Categories)


LabelEncoder()

In [24]:
X_train['Title'] = le_Title.transform(X_train['Title'])

X_train['Edition_Type'] = le_Edition_Type.transform(X_train['Edition_Type'])



X_train['Edition_Month'] = le_Edition_Month.transform(X_train['Edition_Month'])

X_train['Author1'] = le_Author.transform(X_train['Author1'])
X_train['Author2'] = le_Author.transform(X_train['Author2'])
X_train['Author3'] = le_Author.transform(X_train['Author3'])
X_train['Author4'] = le_Author.transform(X_train['Author4'])
X_train['Author5'] = le_Author.transform(X_train['Author5'])
X_train['Author6'] = le_Author.transform(X_train['Author6'])
X_train['Author7'] = le_Author.transform(X_train['Author7'])


X_train['Genre1'] = le_Genre.transform(X_train['Genre1'])
X_train['Genre2'] = le_Genre.transform(X_train['Genre2'])


X_train['Category1'] = le_Category.transform(X_train['Category1'])
X_train['Category2'] = le_Category.transform(X_train['Category2'])


In [25]:
X_train.head()


,Title,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Edition_Type,Edition_Month,Edition_Year,Ratings,Reviews,Genre1,Genre2,Category1,Category2
0,5802,797,3073,3073,3073,3073,3073,3073,13,7,2016,4.0,8,0,267,0,12
1,2120,391,3073,3073,3073,3073,3073,3073,13,10,2012,3.9,14,80,267,2,6
2,2984,4353,3073,3073,3073,3073,3073,3073,13,3,1982,4.8,6,211,267,8,12
3,189,78,3073,3073,3073,3073,3073,3073,13,11,2017,4.1,13,98,267,5,16
4,2987,1221,3073,3073,3073,3073,3073,3073,8,11,2006,5.0,1,284,267,1,7


In [26]:
X_test['Title'] = le_Title.transform(X_test['Title'])

X_test['Edition_Type'] = le_Edition_Type.transform(X_test['Edition_Type'])



X_test['Edition_Month'] = le_Edition_Month.transform(X_test['Edition_Month'])

X_test['Author1'] = le_Author.transform(X_test['Author1'])
X_test['Author2'] = le_Author.transform(X_test['Author2'])
X_test['Author3'] = le_Author.transform(X_test['Author3'])
X_test['Author4'] = le_Author.transform(X_test['Author4'])
X_test['Author5'] = le_Author.transform(X_test['Author5'])
X_test['Author6'] = le_Author.transform(X_test['Author6'])
X_test['Author7'] = le_Author.transform(X_test['Author7'])


X_test['Genre1'] = le_Genre.transform(X_test['Genre1'])
X_test['Genre2'] = le_Genre.transform(X_test['Genre2'])


X_test['Category1'] = le_Category.transform(X_test['Category1'])
X_test['Category2'] = le_Category.transform(X_test['Category2'])


In [27]:
X_test.head()


,Title,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Edition_Type,Edition_Month,Edition_Year,Ratings,Reviews,Genre1,Genre2,Category1,Category2
0,5082,4058,3073,3073,3073,3073,3073,3073,12,11,1986,4.4,960,324,267,5,16
1,2906,1401,3073,3073,3073,3073,3073,3073,13,0,2018,5.0,1,273,267,4,9
2,751,949,3073,3073,3073,3073,3073,3073,13,7,2011,5.0,4,314,267,14,12
3,6232,169,3073,3073,3073,3073,3073,3073,13,9,2016,4.1,11,295,267,4,9
4,3790,3505,3073,3073,3073,3073,3073,3073,13,2,2011,4.4,9,235,267,10,11


In [28]:
# Feature Scaling
#________________

from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)


Y_train = Y_train.reshape((len(Y_train), 1)) 

Y_train = sc.fit_transform(Y_train)

Y_train = Y_train.ravel()


In [29]:
X_train.shape #SC


(6237, 17)

In [30]:
Y_train.shape #SC


(6237,)

In [31]:
X_train


array([[ 1.22489462, -1.11014305,  0.10361038, ..., -0.04054244,
        -1.21387465,  0.3167796 ],
       [-0.64983923, -1.40876249,  0.10361038, ..., -0.04054244,
        -0.82060991, -1.88135243],
       [-0.20992341,  1.50535127,  0.10361038, ..., -0.04054244,
         0.35918432,  0.3167796 ],
       ...,
       [ 0.91379674, -0.11278358,  0.10361038, ..., -0.04054244,
         1.53897855,  0.3167796 ],
       [-0.75676322, -1.56395633,  0.10361038, ..., -0.04054244,
        -1.21387465,  0.3167796 ],
       [ 0.95096556, -0.2951915 ,  0.10361038, ..., -0.04054244,
        -1.21387465,  0.3167796 ]])

In [32]:
X_test


array([[ 0.8582981 ,  1.2883741 ,  0.10361038, ..., -0.04054244,
        -0.23071279,  1.78220095],
       [-0.24963804, -0.66589149,  0.10361038, ..., -0.04054244,
        -0.42734517, -0.78228642],
       [-1.34688178, -0.99834465,  0.10361038, ..., -0.04054244,
         1.53897855,  0.3167796 ],
       ...,
       [ 1.06145367,  0.00342793,  0.10361038, ..., -0.04054244,
         0.35918432,  0.3167796 ],
       [ 0.20911677, -0.49672284,  0.10361038, ..., -0.04054244,
        -0.82060991, -1.88135243],
       [-1.15085447, -1.35139225,  0.10361038, ..., -0.04054244,
         0.75244907, -0.04957574]])

In [33]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 123)


In [34]:
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)


(5613, 17)
(5613,)
(624, 17)
(624,)


In [35]:
!pip install xgboost


In [36]:
from xgboost import XGBRegressor
import numpy as np

xgb=XGBRegressor( objective='reg:squarederror', max_depth=6, learning_rate=0.1, n_estimators=100, booster = 'gbtree', n_jobs = -1,random_state = 1)
xgb.fit(train_x,train_y)

y_pred = sc.inverse_transform(xgb.predict(val_x))
y_true = sc.inverse_transform(val_y)

error = np.square(np.log10(y_pred +1) - np.log10(y_true +1)).mean() ** 0.5
score = 1 - error

print("RMLSE Score = ", score)


RMLSE Score =  0.7163958911417463


In [37]:
# Fitting the complete training set (inclusing val_x and val_y)
xgb.fit(X_train,Y_train)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1,
             objective='reg:squarederror', random_state=1, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [38]:
# Predicting for test set
y_pred_xgb = sc.inverse_transform(xgb.predict(X_test))


In [39]:
solution = pd.DataFrame(y_pred_xgb, columns = ['Price'])
solution.to_excel('Predict_Book_Price_Soln.xlsx', index = False)


In [40]:
solution.head(10)


,Price
0,214.681122
1,1330.917114
2,624.322449
3,844.769043
4,425.502533
5,890.277893
6,956.208252
7,339.453979
8,558.580017
9,467.108673


In [41]:
!pip install bayesian-optimization


In [42]:
from bayes_opt import BayesianOptimization
import xgboost as xgb
#from sklearn.metrics import mean_squared_error,mean_squared_log_error


In [43]:
dtrain = xgb.DMatrix(X_train, label= Y_train)


In [44]:
def bo_tune_xgb(max_depth, gamma, n_estimators ,learning_rate):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,      
              'n_estimators': int(n_estimators),
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'rmse'}
    
    #Cross validating with the specified parameters in 5 folds and 70 iterations
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=10)    
    
    #Return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]


In [45]:
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (1, 300), 
                                             'gamma': (0, 1),
                                             'learning_rate':(0,1),
                                             'n_estimators':(1,1000)
                                            })


xgb_bo.maximize(n_iter=10, init_points=10, acq='ei')


|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------
[13:10:27] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:10:27] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:10:27] WARNING: C:\Users\Administrator\workspace\xgboost-win64_relea


|  3        | -0.9689   |  0.02857  |  0.2407   |  194.7    |  892.7    |
[13:11:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:11:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:11:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be u


[13:12:40] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


|  6        | -1.07     |  0.4089   |  0.7055   |  217.7    |  470.9    |
[13:13:09] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:13:09] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be u

|  8        | -1.062    |  0.437    |  0.6841   |  150.3    |  394.1    |
[13:14:07] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:14:07] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:14:07] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be us


[13:15:08] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:08] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:08] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in l

|  13       | -1.112    |  1.0      |  0.0      |  1.0      |  676.0    |
[13:15:58] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:58] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:15:58] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be us

|  16       | -0.9092   |  0.09661  |  0.1338   |  1.113    |  13.19    |
[13:16:57] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:16:57] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:16:57] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be us

[13:17:18] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:17:18] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


|  19       | -1.055    |  0.7842   |  0.6756   |  128.0    |  2.537    |
[13:17:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be us

In [46]:
#Extracting the best parameters
params = xgb_bo.max['params']

print(params)

#Conversting the max_depth and n_estimator values from float to int
params['max_depth']= int(round(params['max_depth']))
params['n_estimators']= int(round(params['n_estimators']))

print(params)


{'gamma': 0.19053446501997284, 'learning_rate': 0.01633336278992714, 'max_depth': 223.87822181484088, 'n_estimators': 732.6569565926101}
{'gamma': 0.19053446501997284, 'learning_rate': 0.01633336278992714, 'max_depth': 224, 'n_estimators': 733}


In [47]:
#Initialize an XGB with the tuned parameters and fit the training data
from xgboost import XGBRegressor
reg = XGBRegressor(**params).fit(X_train,Y_train)

y_pred_reg = sc.inverse_transform(reg.predict(X_test))


In [48]:
solution_bo = pd.DataFrame(y_pred_reg, columns = ['Price'])

solution_bo.head(10)


,Price
0,242.566833
1,1796.207275
2,317.815979
3,1187.809570
4,387.378754
5,569.480408
6,614.862061
7,316.659302
8,390.683502
9,463.463379


In [49]:
solution_bo.to_excel('Predict_Book_Prices_BO_Soln.xlsx', index = False)


In [50]:
y_pred_reg


array([ 242.56683, 1796.2073 ,  317.81598, ...,  629.0245 ,  273.74896,
        598.17194], dtype=float32)

In [60]:
score


0.7163958911417463